In [27]:
%matplotlib inline
######################################################################################################
#
#         SCRIPT DESCRIPTION: Basic Q-Learning in a single intersection
#
#         Input: Map name, route file
#         Output: Tripinfo.xml
#
#####################################################################################################

# Set OPERATING SYSTEM
OS = "windows"
# Set MAP and demand
used_map = 'triple_straight_random'
demand = "random"
# Select whether to color code the vehicles
color_vehicles = True
persistent_color = False
round_robin_flag = True
queue_tracking = True
positional_tracking = True
tracking_length = 36000
simulation_length = 36000
# Select Memory
reloadQm = True
saveQm = False

# Set Path for Sumo Home folder
if OS == "iOS":
    sumo_home_dir = "/Users/acabrejasegea/sumo-0.32/"
elif OS == "windows":
    sumo_home_dir = "C:\Program Files (x86)\DLR\Sumo"
else:
    sys.exit("Please declare a supported operative system")
    
# Create storage if needed
if positional_tracking == True:
    tracking_data = [[] for _ in range(tracking_length)]
if queue_tracking == True:
    combined_queue = []

# Import basic libraries
import os
import sys
import optparse
import subprocess
import random
import random
import pdb
import xmltodict
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from __future__ import absolute_import
from __future__ import print_function
import json

# Current working directory
working_dir = os.getcwd()

# Set environment variable SUMO_HOME
try:
    # Use this line only to manually define SUMO_HOME path
    os.environ["SUMO_HOME"] = sumo_home_dir
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
except ImportError:
    sys.exit(
        "Please declare environment variable 'SUMO_HOME' as the root directory of your sumo installation (it should contain folders 'bin', 'tools' and 'docs')")

# Import API libraries
from sumolib import checkBinary
import traci

In [28]:
%%javascript
IPython.notebook.kernel.execute('notebook_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [29]:
# Set important directories
if OS=="iOS":
    networks_dir = "/Users/acabrejasegea/sumo-0.32/Data/Networks/"
    scripts_dir = "/Users/acabrejasegea/sumo-0.32/tools/"
    runners_dir = "/Users/acabrejasegea/Desktop/ATI_Project/MLforFlowOptimisation/"
elif OS=="windows":
    networks_dir = "C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Networks"
    scripts_dir = "C:\\Program Files (x86)\\DLR\\Sumo\\tools\\"
    runners_dir = "C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Runners"
else:
    sys.exit("Please declare a supported operative system")
notebook = (os.path.splitext(notebook_name))[0]

In [30]:
# Creates a route file for the simulation
def generate_routefile(simulation_length):
    random.seed(17)  # make tests reproducible
    time_counter = 0
    # demand per second from different directions
    if demand == "fixed":
        pWE  = 1. / 8
        pEW  = 1. / 8
        pNS1 = 1. / 20
        pNS2 = 1. / 20
        pNS3 = 1. / 20
        pSN1 = 1. / 20
        pSN2 = 1. / 20
        pSN3 = 1. / 20
    elif demand == "random":
        thresholds = np.linspace(0,simulation_length,6)
        pWE_v  = np.linspace(0.04, 0.16, 5)
        pEW_v  = np.linspace(0.04, 0.16, 5)
        pNS1_v = np.linspace(0.02, 0.08, 5)
        pNS2_v = np.linspace(0.02, 0.08, 5)
        pNS3_v = np.linspace(0.02, 0.08, 5)
        pSN1_v = np.linspace(0.02, 0.08, 5)
        pSN2_v = np.linspace(0.02, 0.08, 5)
        pSN3_v = np.linspace(0.02, 0.08, 5)
    else:
        sys.exit("Error in kind of demand, please choose random or fixed")
    
    
    # Algorithmic access to routefile (names need to be consistent)
    with open(os.path.join(networks_dir, used_map, (used_map + '.rou.xml')), "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2" maxSpeed="25" guiShape="passenger"/>

        <route id="right" edges="iW iCW oCE oE" />
        <route id="left" edges="iE iCE oCW oW" />
        <route id="down1" edges="iN1 oS1" />
        <route id="down2" edges="iN2 oS2" />
        <route id="down3" edges="iN3 oS3" />
        <route id="up1" edges="iS1 oN1" />
        <route id="up2" edges="iS2 oN2" />
        <route id="up3" edges="iS3 oN3" />""", file=routes)
        lastVeh = 0
        vehNr = 0
        for i in range(simulation_length):
            if demand == "random":
                if i in thresholds:
                    pWE  = random.choice(pWE_v)
                    pEW  = random.choice(pEW_v)
                    pNS1 = random.choice(pNS1_v)
                    pNS2 = random.choice(pNS2_v)
                    pNS3 = random.choice(pNS3_v)
                    pSN1 = random.choice(pSN1_v)
                    pSN2 = random.choice(pSN2_v)
                    pSN3 = random.choice(pSN3_v)
            
            if random.uniform(0, 1) < pWE:
                print('    <vehicle id="right_%i" type="typeWE" route="right" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pEW:
                print('    <vehicle id="left_%i" type="typeWE" route="left" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS1:
                print('    <vehicle id="down_%i" type="typeNS" route="down1" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS2:
                print('    <vehicle id="down_%i" type="typeNS" route="down2" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pNS3:
                print('    <vehicle id="down_%i" type="typeNS" route="down3" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN1:
                print('    <vehicle id="down_%i" type="typeNS" route="up1" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN2:
                print('    <vehicle id="down_%i" type="typeNS" route="up2" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
            if random.uniform(0, 1) < pSN3:
                print('    <vehicle id="down_%i" type="typeNS" route="up3" depart="%i" color="0,1,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
                lastVeh = i
        print("</routes>", file=routes)

In [31]:
class NetworkParser:

    def __init__(self, file_path):
        self.junctions = dict()
        self.junction_lights = dict()

        with open(file_path) as f:
            self.file = xmltodict.parse(f.read())

    def get_lane_ids(self):
        for junction in self.file['net']['junction']:
            if junction['@type'] == 'traffic_light':
                self.junctions[junction['@id']] = junction['@incLanes'].split(' ')
        return self.junctions

    # Given a LaneID it returns the traffic light ID
    def get_trafficlights_by_laneID(self, laneID):
        for connection in self.file['net']['connection']:
            if connection['@from'] == laneID.split('_')[0] and '@tl' in connection:
                return connection['@tl']

    def get_junction_trafficlights(self):
        # If the dictionarty for junctions is empty
        if not self.junctions:
            self.get_lane_ids()
        for junctionID in self.junctions.keys():
            for lane in self.junctions[junctionID]:
                traffic_light = self.get_trafficlights_by_laneID(lane)
                if traffic_light in self.junction_lights and lane not in self.junction_lights[traffic_light]:
                    self.junction_lights[traffic_light].append(lane)
                else:
                    self.junction_lights[traffic_light] = []
        return self.junction_lights

    def get_traffic_light(self, laneID):
        if not self.junction_lights:
            self.get_junction_trafficlights()
        for traffic_light, lanes in self.junction_lights.iteritems():
            if laneID in lanes:
                return traffic_light

    def get_phases(self, traffic_light):
        phases = []
        z = 0
        for tls in self.file['net']['tlLogic']:
            if z < 1:
                a=0
            z+=1
            if tls['@id'] == traffic_light:
                for tls_phase in tls['phase']:
                    phases.append(tls_phase)
        return phases

In [32]:
class Qlearner:
    def __init__(self,TLid,lanes,phases):
        self.jid     = TLid
        self.laneids = lanes
        self.Nlanes  = len(self.laneids)                                #len(nodes.junctions[self.jid])
        self.phases  = range(0, 23)                            #traci.trafficlights.getallPhases
        num_phases   = len(self.phases)
        self.Qm      = np.zeros((275, 11))
        
        self.state   = 0
        self.action  = 0
        self.neighbor_state = 0
                                
        self.cycle_time = 90
        
        self.historic_reward = list()
        self.historic_queues = [[] for x in range(self.Nlanes)]
  
    def set_action(self,action, ):
        traci.trafficlights.setProgram(self.jid, str(action))
                                                                

    def get_reward(self):
        # Reward defined as a measure of the amount of waiting in the intersection
        return -sum([traci.lane.getLastStepHaltingNumber(lane_id) for lane_id in self.laneids])
 

    def get_state(self,threshold_l=3):
        # State is defined as a combination b/w 5 load level for the 2 routes
        NS_occupancy = traci.lane.getLastStepHaltingNumber(self.laneids[0]) + traci.lane.getLastStepHaltingNumber(self.laneids[2])
        EW_occupancy = traci.lane.getLastStepHaltingNumber(self.laneids[1]) + traci.lane.getLastStepHaltingNumber(self.laneids[3])

        NS_state = np.trunc(NS_occupancy/3)
        EW_state = np.trunc(EW_occupancy/3)
        
        if NS_state > 4:
            NS_state = 4
        if EW_state >4:
            EW_state = 4
        
        if self.jid == learners[0].jid:
            self.neighbor_state = 0
        elif self.jid == learners[1].jid:
            self.neighbor_state = traci.trafficlights.getProgram("Centre1")
        elif self.jid == learners[2].jid:
            self.neighbor_state = traci.trafficlights.getProgram("Centre2")
        else:
            sys.exit("Error in getProgram or self.jid")
        
        state = int(NS_state + 5* EW_state + 25* np.float64(self.neighbor_state))

        return(state)

    def update(self,gamma=0.8,alpha=0.95,epsilon=0.025):
        
        # Get current state, action from class
        state    = self.state
        action   = self.action
        # Generate new action and the reward
        newstate = self.get_state()
        reward   = self.get_reward()
        self.historic_reward.append(reward)

        # Update the Q matrix
        self.Qm[state, action] = self.Qm[state, action]*(1-alpha) + alpha*(reward + gamma*np.max(self.Qm[newstate,:]))

        # Explore or exploit
        r=np.random.rand(1)
        if r > epsilon :
            newaction = np.argmax(self.Qm[state,:])
        else:
            newaction = np.random.randint(0, 10)
    
        # Set variables for next loop and reset cycle timer
        self.state = newstate
        self.action = newaction
                                
        self.set_action(newaction)

In [33]:
def run(learners = []):
    cycle_timer = 0
    timer = 0
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        timer = timer + 1
        if cycle_timer == 90:
            for learner in learners:
                learner.update()
            cycle_timer = 0
        else:
            cycle_timer = cycle_timer + 1    
        if queue_tracking == True:
            combined_queue.append(traci.lane.getLastStepHaltingNumber(learners[0].laneids[0])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[1])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[2])+
                                     traci.lane.getLastStepHaltingNumber(learners[0].laneids[3])+
                                     traci.lane.getLastStepHaltingNumber(learners[1].laneids[0])+
                                     traci.lane.getLastStepHaltingNumber(learners[1].laneids[1])+
                                     traci.lane.getLastStepHaltingNumber(learners[1].laneids[2])+
                                     traci.lane.getLastStepHaltingNumber(learners[1].laneids[3])+
                                     traci.lane.getLastStepHaltingNumber(learners[2].laneids[0])+
                                     traci.lane.getLastStepHaltingNumber(learners[2].laneids[1])+
                                     traci.lane.getLastStepHaltingNumber(learners[2].laneids[2])+
                                     traci.lane.getLastStepHaltingNumber(learners[2].laneids[3]))
            for j in range(3):
                for k in range(4):
                    learners[j].historic_queues[k].append(traci.lane.getLastStepHaltingNumber(learners[j].laneids[k]))
            
        if positional_tracking == True:
            vehicles = traci.vehicle.getIDList()
            for vehicle in vehicles:
                tracking_data[timer].append(tuple((vehicle, 
                                                    traci.vehicle.getPosition(vehicle),
                                                    traci.vehicle.getCO2Emission(vehicle),
                                                    traci.vehicle.getNOxEmission(vehicle))))
        
        if color_vehicles == True:
            vehicleIDs = traci.vehicle.getIDList()
            waiting_time = list()
            for vehicle in vehicleIDs:
                wait = traci.vehicle.getWaitingTime(vehicle)
                if wait >= 45:
                    traci.vehicle.setColor(vehicle,(255,0,0,255))
                elif wait >= 20:
                    traci.vehicle.setColor(vehicle,(255,165,0,255)) 
                elif wait >= 1:
                    traci.vehicle.setColor(vehicle,(255,255,0,255))
                if persistent_color == False:
                    if wait < 1:
                        traci.vehicle.setColor(vehicle,(0,255,0,255))

    
    sys.stdout.flush()

### ORIGINAL SIMPLE CONTROLLER BELOW ###
# The program looks like this
#    <tlLogic id="0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="31" state="GrGr"/>
#        <phase duration="6"  state="yryr"/>
#        <phase duration="31" state="rGrG"/>
#        <phase duration="6"  state="ryry"/>
#    </tlLogic>
#
#def run():
#    """execute the TraCI control loop"""
#    step = 0
#    # we start with phase 2 where EW has green
#    traci.trafficlight.setPhase("0", 2)
#    while traci.simulation.getMinExpectedNumber() > 0:
#        traci.simulationStep()
#        if traci.trafficlight.getPhase("0") == 2:
#            # we are not already switching
#            if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
#                # there is a vehicle from the north, switch
#                traci.trafficlight.setPhase("0", 3)
#            else:
#                # otherwise try to keep green for EW
#                traci.trafficlight.setPhase("0", 2)
#        step += 1
#    traci.close()
#    sys.stdout.flush()

In [34]:
def get_options():
    optParser = optparse.OptionParser()
    optParser.add_option("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
    options, args = optParser.parse_args()
    return options

In [35]:
# Deployment of Network Parser (input is network file)
npa = NetworkParser(os.path.join(networks_dir, used_map, (used_map + '.net.xml')))

In [36]:
nodes = npa.junctions
phase_list = [len(npa.get_phases(k)) for k in npa.get_junction_trafficlights().keys()]
node_list = list(nodes.keys())

In [37]:
# Checks
print(nodes)
print(nodes.keys())
print(phase_list)

{'Centre1': ['iN1_0', 'oCW_0', 'iS1_0', 'iW_0'], 'Centre2': ['iN2_0', 'iCE_0', 'iS2_0', 'iCW_0'], 'Centre3': ['iN3_0', 'iE_0', 'iS3_0', 'oCE_0']}
dict_keys(['Centre1', 'Centre2', 'Centre3'])
[44, 44, 44]


In [ ]:
generate_routefile(simulation_length)

In [38]:
sumoBinary = checkBinary('sumo-gui')
# OR
#sumoBinary = checkBinary('sumo')

In [39]:
learners=[Qlearner(ID,lan,p) for ID, lan, p in zip(nodes.keys(),nodes.values(),phase_list)]

In [40]:
if reloadQm == True:
    for i in range(len(nodes.keys())):
        if OS == "iOS":
            learners[i].Qm = np.load(networks_dir+used_map+"/"+"Qm_" + str(i)+ demand  + ".npy")
            print("Loaded pre-trained Q-matrix from file: "+'\n'+networks_dir+used_map+"/"+"Qm_" + str(i)+ demand  + ".npy")
            with open(networks_dir+used_map+"/"+"Reward_" + str(i) + demand  + ".pickle", 'rb') as fp:
                learners[i].historic_reward = pickle.load(fp)
            print("Loaded Reward history to file: "+networks_dir+used_map+"/"+"Reward_" + str(i) + demand  + ".pickle")
        elif OS == "windows":
            learners[i].Qm = np.load(networks_dir+"\\"+used_map+"\\"+"Qm_" + str(i)+ demand  + ".npy")
            print("Loaded pre-trained Q-matrix from file: "+'\n'+networks_dir+used_map+"\\"+"Qm_" + str(i)+ demand  + ".npy")
            #with open(networks_dir+"\\"+used_map+"\\"+"Reward_" + str(i) + demand  + ".pickle", 'rb') as fp:
            #    learners[i].historic_reward = pickle.load(fp)
            #print("Loaded Reward history to file: "+networks_dir+"\\"+used_map+"\\"+"Reward_" + str(i) + demand  + ".pickle")

Loaded pre-trained Q-matrix from file: 
C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Networkstriple_straight_random\Qm_0random.npy
Loaded pre-trained Q-matrix from file: 
C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Networkstriple_straight_random\Qm_1random.npy
Loaded pre-trained Q-matrix from file: 
C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Networkstriple_straight_random\Qm_2random.npy


In [41]:
traci.start([sumoBinary, "-c", os.path.join(networks_dir, used_map, (used_map + '.sumocfg')),
                             "--tripinfo-output", os.path.join((notebook + '_out'), 'tripinfo.xml')])

(17, 'SUMO 0.32.0')

In [42]:
run(learners)

C:\Program Files (x86)\DLR\Sumo\tools\traci\domain.py:112: UserWarning: The domain trafficlights is deprecated, use trafficlight instead.
  self._name, self._deprecatedFor))  # , DeprecationWarning)


FatalTraCIError: connection closed by SUMO

In [ ]:
if saveQm == True:
    for i in range(len(nodes.keys())):
        if OS == "iOS":
            np.save(networks_dir+used_map+"/"+"Qm_" + str(i) + demand + ".npy", learners[i].Qm)
            print("Saved Q-matrix to file: "+'\n'+networks_dir+used_map+"/"+"Qm_" + str(i) + demand + ".npy")
            with open(networks_dir+used_map+"/"+"Reward_" + str(i) + demand + ".pickle", 'wb') as fp:
                pickle.dump(learners[i].historic_reward, fp)
            print("Saved Reward history to file: "+'\n'+networks_dir+used_map+"/"+"Reward_" + str(i) + demand + ".pickle")
        elif OS == "windows":
            np.save(networks_dir+"\\"+used_map+"\\"+"Qm_" + str(i) + demand + ".npy", learners[i].Qm)
            print("Saved Q-matrix to file: "+'\n'+networks_dir+"\\"+used_map+"\\"+"Qm_" + str(i) + demand + ".npy")
            with open(networks_dir+"\\"+used_map+"\\"+"Reward_" + str(i) + demand + ".pickle", 'wb') as fp:
                pickle.dump(learners[i].historic_reward, fp)
            print("Saved Reward history to file: "+'\n'+networks_dir+"\\"+used_map+"\\"+"Reward_" + str(i) + demand + ".pickle")

In [ ]:
learners[1].Qm

In [ ]:
# PLOT GLOBAL TRAINING
plt.figure(figsize=(20,10))
plot_series = [x+y+z for x,y,z in zip(learners[0].historic_reward,learners[1].historic_reward,learners[2].historic_reward)]
plt.plot(plot_series[0:5000])

xcoords = [0,1000,2000,3000,4000,5000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')
    
texts = ['Demand Regime 1', 'Demand Regime 2', 'Demand Regime 3', 'Demand Regime 4', 'Demand Regime 5' ]
for i in range(len(texts)):
    plt.text(xcoords[i]+160,-100,texts[i], fontsize = 14, color='red')

plt.title("Instantaneous reward over time, "+r'$\alpha = 0.95, $' + r'$\epsilon=0.025$', fontsize = 20)
plt.xlabel("Time [cycles]"       , fontsize=16)
plt.ylabel("Instantaneous Reward", fontsize=16)

savedir = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\07_End_of_Internship_Presentation\\'
filename = 'trained_global_'+used_map+'_untrained.png'
plt.savefig(savedir+filename)

In [ ]:
# PLOT TRAINING PER LEARNER
chosen_learner = 2
plt.figure(figsize=(20,10))
plot_series = learners[chosen_learner].historic_reward
plt.plot(plot_series[0:5000])

xcoords = [0,1000,2000,3000,4000,5000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')
    
texts = ['Demand Regime 1', 'Demand Regime 2', 'Demand Regime 3', 'Demand Regime 4', 'Demand Regime 5' ]
for i in range(len(texts)):
    plt.text(xcoords[i]+160,-53,texts[i], fontsize = 14, color='red')

plt.title("Instantaneous reward over time in Learner "+str(chosen_learner)+", "+r'$\alpha = 0.95, $' + r'$\epsilon=0.025$', fontsize = 20)
plt.xlabel("Time [cycles]"       , fontsize=16)
plt.ylabel("Instantaneous Reward", fontsize=16)
plt.ylim(-55,0)

savedir = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\07_End_of_Internship_Presentation\\'
filename = 'trained_partial_'+used_map+'_untrained_learner_'+str(chosen_learner)+'.png'
plt.savefig(savedir+filename)

In [ ]:
trained_cycle = combined_queue[0:90000]
#untrained_cycle = combined_queue

In [ ]:
trained_cycle = trained_cycle[0:90000]
untrained_cycle = untrained_cycle[0:90000]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(trained_cycle[0:15000], color = 'blue', label = "Trained Controller")

 
plt.plot(untrained_cycle[0:15000], color = 'red', label = "Untrained Controller")
plt.legend(prop={'size' : 16})

plt.title("Total stopped vehicles in the model", fontsize = 20)
plt.xlabel("Time [seconds]"       , fontsize=16)
plt.ylabel("Aggregated queue length [vehicles]", fontsize=16)

In [ ]:
## PLOT FOR QUEUES
chosen_learner = 2
plt.figure(figsize=(20,20))
y1 = learners[chosen_learner].historic_queues[0][0:450000]
y2 = learners[chosen_learner].historic_queues[1][0:450000]
y3 = learners[chosen_learner].historic_queues[2][0:450000]
y4 = learners[chosen_learner].historic_queues[3][0:450000]

x1 = range(len(y1))
x2 = range(len(y1))
x3 = range(len(y1))
x4 = range(len(y1))

plt.subplot(4, 1, 1)
plt.plot(x1, y1, '-')
plt.ylim(0,25)
plt.title('Queues in learner '+str(chosen_learner))
plt.ylabel('North Queue Length')
xcoords = [0,90000,180000,270000,360000,450000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')
    
texts = ['Demand Regime 1', 'Demand Regime 2', 'Demand Regime 3', 'Demand Regime 4', 'Demand Regime 5' ]
for i in range(len(texts)):
    plt.text(xcoords[i]+16000,23,texts[i], fontsize = 14, color='red')
    

plt.subplot(4, 1, 2)
plt.plot(x2, y2, '-')
plt.ylim(0,25)
plt.ylabel('West Queue Length')
xcoords = [0,90000,180000,270000,360000,450000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')

plt.subplot(4, 1, 3)
plt.plot(x1, y1, '-')
plt.ylim(0,25)
plt.ylabel('South Queue Length')
xcoords = [0,90000,180000,270000,360000,450000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')

plt.subplot(4, 1, 4)
plt.plot(x2, y2, '-')
plt.ylim(0,25)
plt.xlabel('Time [s]')
plt.ylabel('East Queue Length')
xcoords = [0,90000,180000,270000,360000,450000]
for xc in xcoords:
    plt.axvline(x=xc, color='red')

savedir = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\07_End_of_Internship_Presentation\\'
filename = 'queues_'+used_map+'_trained_learner_'+str(chosen_learner)+'.png'
plt.savefig(savedir+filename)

plt.show()

In [ ]:
## Exporting Positional data as JSON
with open(used_map+'_timesteps_'+str(tracking_length)+'.json', 'w') as outfile:
    json.dump(positional_data, outfile)

In [ ]:
a = json.loads(output)
a